In [19]:
from CircularBuffer import CircularBuffer
from CircularBuffer import StereoDelay
from scipy.io import wavfile
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import soundfile as sf

#KYW inst
#fs, data = wavfile.read('/Users/kevinbrennan/Desktop/Final Project/Music-Technology-Final-Project/Sound Examples/KYW Inst.wav')
#ATF Drums
fs, data = wavfile.read('/Users/kevinbrennan/Desktop/Final Project/Music-Technology-Final-Project/Sound Examples/Around the fur drums.wav')

# Get the bit depth of the audio data (assumes audio_data is a NumPy array)
bit_depth = data.dtype.itemsize * 8

# Calculate the scaling factor for normalization
scaling_factor = 2 ** (bit_depth - 1)  # For signed audio

# Convert audio data to floating-point values and normalize
data = data.astype(np.float32) / scaling_factor

# Verify that the data is now in the range -1 to 1
print(f"Min Value: {np.min(data)}")
print(f"Max Value: {np.max(data)}")

Min Value: -1.0
Max Value: 0.999969482421875


In [20]:
#### Stereo Delay ####

delay_ms = 80 #Delay time in ms
wet_amt = 0.5 #Wet amount (0 -> 1)
fdbk_amt = 0.4 #Feedback amount (0 -> 1)

# Delay time in samples for creating buffer
delaySamps = ((delay_ms/1000) * fs)

# Stereo buffer setup with differing delay amounts for each side
stereoBuffer = StereoDelay(delaySamps, delaySamps+100)

In [21]:
# Audio output array
stereo_audio_out = np.zeros((len(data) + stereoBuffer.buffer_maxLen(), 2))

# Processing Loop
for i in range(len(stereo_audio_out)):
    # Take next wet sample from buffer
    delayed_s = stereoBuffer.get_samples_from_buffer()

    feedbackSamps = np.multiply(delayed_s, fdbk_amt)
    
    # Push next dry sample to buffer (if there are dry samples left)
    if(i < len(data)):
        stereoBuffer.push_samples_to_buffer((data[i] + feedbackSamps))

    # Combine with dry signal and put in output array
    stereo_audio_out[i] = np.multiply(wet_amt,delayed_s)

    # Include dry if there is dry left
    if(i < len(data)):
        stereo_audio_out[i] += np.multiply((1 - wet_amt), data[i])


In [22]:
sd.play(stereo_audio_out, fs)

In [23]:
#Write to wav file
#sf.write('Delay_ATF.wav', stereo_audio_out, fs)